### launching of SPs



In [7]:
import pyodbc
from sql.config import SQL_SERVER, DB_NAME
import sql.naming_convention as nc

conn_str = (
            f'DRIVER={{ODBC Driver 17 for SQL Server}};'
            f'SERVER={SQL_SERVER};'
            f'DATABASE={DB_NAME};'
            f'Trusted_Connection=yes;'
            f'Encrypt=yes;'
            f'TrustServerCertificate=yes;'
        )

print('connecting...')
sql_connection = pyodbc.connect(conn_str, timeout=60)
print("Connection successful!")

connecting...
Connection successful!


In [8]:
entity_name = 'Symbol_IBESTickerEnhanced'
sp_name = nc.pull_sp_name(nc.default_rename(entity_name))
exec_stm = f"EXEC {sp_name}"
cursor = sql_connection.cursor()
print(f'trying to run : {exec_stm}')
cursor.execute(exec_stm)


trying to run : EXEC dbo.PullData_Symbol_IBESTickerEnhanced2_prc
